In [1]:
import sys
sys.path.append(r"../../utils")

import pandas as pd
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, precision_recall_curve, roc_curve, auc, average_precision_score

from questionnaires_aggregation import c_ssrs_aggregation
from utils import impute_from_column, simple_eda
from questions_columns import c_ssrs, sci_af_ca, mfq, sci_af_ca, sci_mother, scs_clin, siq, sdq, c_ssrs_intake, scared, ATHENS, SAS, c_ssrs_clin, demographics_m
from xgboost import XGBClassifier, XGBRegressor

In [2]:
# Splitting the data into train, validation, and test sets
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2
c_ssrs = c_ssrs[:-3]
target_col = 'suicidal_behavior_time2'

sci_cols = ['is_sci_af_ca_missing', 'sci_af_ca_Factor1', 'sci_af_ca_Factor2',
             'sci_af_ca_Factor3', 'sci_af_ca_Factor4', 
             'sci_af_ca_Factor5', 'gender', 'age_child_pre'] + sci_af_ca + mfq + sci_mother + scs_clin+ siq + sdq + c_ssrs_intake + scared + ATHENS + SAS + c_ssrs_clin# + demographics_m

In [3]:
df = pd.read_csv(r"../create_dataset/data_for_prediction_task - measurement time axis.csv")
df_intake = df[df.measurement == 'time1']
df_target = df[df.measurement == 'time2']

In [4]:
df_target = df_target[[target_col, 'id']]
df_intake = df_intake.dropna(subset=sci_af_ca, thresh=20)
df_intake = df_intake[sci_cols + ['id']]
data_for_prediction = pd.merge(df_intake, df_target, on='id', how='inner')


In [5]:
label = data_for_prediction[target_col]
features = data_for_prediction[sci_cols]

In [13]:

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=test_ratio, random_state=420, stratify=label)

# Splitting the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), random_state=420, stratify=y_train)

In [14]:
# Create a CatBoostClassifier object with early stopping based on the validation set
#eval_set  = [(X_val, y_val)]
model = CatBoostClassifier(random_state=420, class_weights = {0: 1, 1: 3}, reg_lambda=10, n_estimators = 20, max_depth= 2)

# Fit the model on the training set
model.fit(X_train, y_train, early_stopping_rounds = 5, verbose=0)#, use_best_model=True)

# Evaluate the model on the training and validation sets

# Make predictions on the training and validation sets
y_train_pred = model.predict(X_train)
y_train_prob = model.predict_proba(X_train)[:, 1]
y_val_pred = model.predict(X_val)
y_val_prob = model.predict_proba(X_val)[:, 1]


y_test_pred = model.predict(X_test)
y_test_prob = model.predict_proba(X_test)[:, 1]


In [15]:
# Function to calculate and print various performance metrics

def print_performance_metrics(y_true, y_pred, y_prob):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_prob)
    average_precision = average_precision_score(y_true, y_prob)
    
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'PR AUC: {average_precision:.4f}')


In [16]:
# Print the performance metrics for the training set
print('Training set performance:')
print_performance_metrics(y_train, y_train_pred, y_train_prob)



Training set performance:
Accuracy: 0.7857
Precision: 0.5526
Recall: 0.7500
ROC AUC: 0.9026
PR AUC: 0.8325


In [17]:
# Print the performance metrics for the validation set
print('Validation set performance:')
print_performance_metrics(y_val, y_val_pred, y_val_prob)


Validation set performance:
Accuracy: 0.7105
Precision: 0.4000
Recall: 0.4444
ROC AUC: 0.6169
PR AUC: 0.3743


In [18]:
# Print the performance metrics for the validation set
print('Test set performance:')
print_performance_metrics(y_test, y_test_pred, y_test_prob)


Test set performance:
Accuracy: 0.6842
Precision: 0.3636
Recall: 0.4444
ROC AUC: 0.6475
PR AUC: 0.4978


In [19]:

# Get feature importance scores
feature_importance = model.get_feature_importance()
feature_names = X_train.columns.tolist()
# Print feature importance scores
print('Feature importance scores:')
for feature_name, score in sorted(zip(feature_names, feature_importance), reverse=1, key = lambda x: x[1]):
    print(f'{feature_name}: {score:.4f}')

Feature importance scores:
siq_3: 23.0600
siq_12: 11.8879
c_ssrs_7_intake: 7.9855
sdq_4: 6.7304
sci_p_1_m: 4.9683
sas_4: 4.8720
sas_3: 4.6589
sci_af_ca_29: 3.9729
athens_7: 3.9379
scared_16: 3.0073
scared_3: 2.7540
sci_c_4_2_clin: 2.6524
c_ssrs_t_2weeks_5_clin: 2.4666
c_ssrs_6_2weeks: 2.2850
sci_af_ca_34: 2.0017
mfq_8: 1.8113
scared_12: 1.4907
sci_p_14_m: 1.4272
sci_c_3_4_clin: 1.3973
sci_af_ca_32: 1.3149
sci_af_ca_33: 1.1415
sas_8: 0.9427
sas_13: 0.9307
scared_21: 0.8240
athens_2: 0.7581
sci_af_ca_26: 0.4525
c_ssrs_t_14_2weeks_clin: 0.1811
sas_11: 0.0455
sdq_16: 0.0226
scared_30: 0.0190
is_sci_af_ca_missing: 0.0000
sci_af_ca_Factor1: 0.0000
sci_af_ca_Factor2: 0.0000
sci_af_ca_Factor3: 0.0000
sci_af_ca_Factor4: 0.0000
sci_af_ca_Factor5: 0.0000
gender: 0.0000
age_child_pre: 0.0000
sci_af_ca_1: 0.0000
sci_af_ca_2: 0.0000
sci_af_ca_3: 0.0000
sci_af_ca_4: 0.0000
sci_af_ca_5: 0.0000
sci_af_ca_6: 0.0000
sci_af_ca_7: 0.0000
sci_af_ca_8: 0.0000
sci_af_ca_9: 0.0000
sci_af_ca_10: 0.0000
sci_af_c

cool, 
use the train-validation-test variables that you created in the last snippet, 
and train CatBoostClassifier, with early_stopping parameter based on the validation set.

after that look at the performance scores on the training & validation sets.
and the feature-importance scores

Since it's a complicated task, we need to use the following metrics: accuracy, precision, recall, precision-recall-auc, roc_auc